


```
# Probability of Default Modeling & Credit Line Simulation  
### CRISP-DM Phase 3: Modeling, Evaluation & Business Impact

This notebook develops a Logistic Regression-based Probability of Default (PD)
model and simulates a dynamic credit limit adjustment strategy to reduce portfolio
Expected Loss.
```



In [3]:
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
dataset_path = '/content/drive/MyDrive/phase2_clean.parquet'

try:
    df = pd.read_parquet(dataset_path)
    print(f"Dataset loaded successfully from: {dataset_path}")
except FileNotFoundError:
    print(f"Error: The file '{dataset_path}' was not found. Please ensure the file is uploaded to your Google Drive and the path is correct.")
except Exception as e:
    print(f"An error occurred while loading the dataset: {e}")

Dataset loaded successfully from: /content/drive/MyDrive/phase2_clean.parquet


now we check for data inconsistency, see if there are any extreme outliers

In [6]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
loan_amnt,1345310.0,14419.972014,8717.050787,500.00,8000.00,12000.000000,20000.00,4.000000e+04
term,1345310.0,41.790196,10.268321,36.00,36.00,36.000000,36.00,6.000000e+01
int_rate,1345310.0,13.239619,4.768716,5.31,9.75,12.740000,15.99,3.099000e+01
installment,1345310.0,438.075533,261.512604,4.93,248.48,375.430000,580.73,1.719830e+03
revol_util,1344453.0,51.810022,24.521072,0.00,33.40,52.200000,70.70,8.923000e+02
revol_bal,1345310.0,16248.114860,22328.168731,0.00,5943.00,11134.000000,19755.75,2.904836e+06
bc_util,1283398.0,59.939572,28.293216,0.00,38.30,63.200000,84.80,3.396000e+02
all_util,537545.0,58.176372,20.935438,0.00,44.00,59.000000,73.00,1.980000e+02
total_rev_hi_lim,1277783.0,32790.937175,36642.589282,0.00,14000.00,24100.000000,40600.00,9.999999e+06
bc_open_to_buy,1284167.0,10186.234200,15350.837521,0.00,1464.00,4700.000000,12383.00,5.599120e+05


In [7]:
(df['revol_util'] > 200).sum()

np.int64(2)

In [8]:
df.loc[df['revol_util'] > 100 , 'revol_util'] = np.nan

In [9]:
df.loc[df['bc_util'] > 100 , 'bc_util'] = np.nan

In [10]:
df.loc[df['all_util'] > 100 , 'all_util'] = np.nan

In [11]:
df.loc[df['total_rev_hi_lim'] == 9999999, 'total_rev_hi_lim'] = np.nan
df.loc[df['tot_hi_cred_lim'] == 9999999, 'tot_hi_cred_lim'] = np.nan

now looking at our EDA and df.describe().T we see that annual income and some other features have some heavy outliers so we check for the 99.9% of the data

In [12]:
df['annual_inc'].quantile(0.999)

np.float64(577382.0000002161)

In [13]:
income_cap = df['annual_inc'].quantile(0.999)

df.loc[df['annual_inc'] > income_cap, 'annual_inc'] = income_cap


In [14]:
df['revol_bal'].quantile(0.999)

np.float64(263117.8930000025)

In [15]:
revol_bal_cap = df['revol_bal'].quantile(0.999)

df.loc[df['revol_bal'] > revol_bal_cap , 'revol_bal'] = revol_bal_cap

In [16]:
(df['annual_inc'] == 0).sum()

np.int64(361)

In [17]:
df.loc[df['annual_inc'] == 0 , 'annual_inc'] = np.nan

In [18]:
df['total_rev_hi_lim'].quantile(0.999)

np.float64(342543.9120000098)

In [19]:
total_limit_cap = df['total_rev_hi_lim'].quantile(0.999)

In [20]:
df.loc[df['total_rev_hi_lim'] > total_limit_cap , 'total_rev_hi_lim'] = np.nan

In [21]:
df.describe(percentiles=[0.9, 0.95, 0.99, 0.999]).T

,count,mean,std,min,50%,90%,95%,99%,99.9%,max
loan_amnt,1345310.0,14419.972014,8717.050787,500.00,12000.000000,28000.000000,32875.000000,35000.000000,4.000000e+04,4.000000e+04
term,1345310.0,41.790196,10.268321,36.00,36.000000,60.000000,60.000000,60.000000,6.000000e+01,6.000000e+01
int_rate,1345310.0,13.239619,4.768716,5.31,12.740000,19.520000,22.150000,26.300000,3.089000e+01,3.099000e+01
installment,1345310.0,438.075533,261.512604,4.93,375.430000,807.920000,963.660000,1221.500000,1.368660e+03,1.719830e+03
revol_util,1339766.0,51.629353,24.359060,0.00,52.000000,84.600000,91.000000,97.500000,9.970000e+01,1.000000e+02
revol_bal,1345310.0,16125.862924,19748.077832,0.00,11134.000000,32425.000000,43374.000000,94553.910000,2.631130e+05,2.631179e+05
bc_util,1262137.0,59.218516,27.961860,0.00,62.400000,94.500000,97.200000,99.300000,1.000000e+02,1.000000e+02
all_util,530951.0,57.540089,20.240626,0.00,59.000000,83.000000,88.000000,97.000000,1.000000e+02,1.000000e+02
total_rev_hi_lim,1276502.0,32326.125362,29789.381564,0.00,24000.000000,64200.000000,85200.000000,151800.000000,2.782499e+05,3.425280e+05
bc_open_to_buy,1284167.0,10186.234200,15350.837521,0.00,4700.000000,26299.400000,38995.700000,74182.680000,1.346504e+05,5.599120e+05


In [22]:
cap = df['delinq_amnt'].quantile(0.999)
df.loc[df['delinq_amnt'] > cap, 'delinq_amnt'] = cap

In [23]:
df['delinq_amnt'].quantile([0.99, 0.995, 0.999])

,delinq_amnt
0.990,0.000000
0.995,0.000000
0.999,1073.864886


In [24]:
# now we freeze the data set

In [25]:
df.drop('loan_status',axis = 1,inplace= True)

In [26]:
df_clean = df.copy()

In [ ]:
df_clean.to_parquet("/content/drive/MyDrive/pd_cleaned.parquet", index=False)

In [28]:
X = df_clean.drop("default", axis=1)
y = df_clean["default"]


In [29]:
y.value_counts(normalize=True)

,proportion
default,
0,0.800374
1,0.199626


After cleaning, separating numerical columns and categorical columns

In [30]:
num_cols = X.select_dtypes(include=np.number).columns
cat_cols = X.select_dtypes(exclude= np.number).columns

print("Numerical:", len(num_cols))
print("Categorical:", len(cat_cols))


Numerical: 45
Categorical: 4


In [31]:
cat_cols

Index(['purpose', 'verification_status', 'home_ownership', 'addr_state'], dtype='object')

##

```
 Preprocessing Pipeline

To prevent data leakage and ensure reproducibility, preprocessing is implemented using a ColumnTransformer inside a Pipeline.

This ensures:

- Imputation is fit only on training data
- Scaling does not use future information
- Feature transformations remain consistent
```




In [32]:
num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])
cat_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])
preprocessor = ColumnTransformer([
    ("numerical",num_pipeline, num_cols),
    ('categorical',cat_pipeline, cat_cols)
])



```
Model Selection: Logistic Regression

Logistic Regression was selected due to:

- Interpretability of coefficients
- Stability across credit portfolios
- Widespread regulatory acceptance in banking
- Direct probability output suitable for Expected Loss calculation

Regularization is applied to reduce overfitting and manage multicollinearity.
```

In [33]:
model = LogisticRegression(
    solver='lbfgs',
    penalty='l2',
    max_iter=1000,
    random_state=42
)

In [34]:
log_pipe = Pipeline([
    ('preprocessor', preprocessor),
    ('model', model)
])




```


 Data Preparation

The cleaned dataset from previous phases is now used for model development.

A stratified train-test split is applied to preserve class proportions due to
inherent default imbalance.


In [35]:
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    stratify=y,
    random_state=42
)

print("Train shape:", X_train.shape)
print("Test shape:", X_test.shape)

Train shape: (1076248, 49)
Test shape: (269062, 49)


In [36]:
X_train_sample = X_train.sample(200000, random_state=42)
y_train_sample = y_train.loc[X_train_sample.index]

In [37]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

Cross Validation for better results on unseen data

In [38]:
skf = StratifiedKFold(n_splits=3, shuffle = True, random_state=42)
log_cross_score = cross_val_score(
    log_pipe,
    X_train_sample,
    y_train_sample,
    cv=skf,
    scoring='roc_auc',
    n_jobs=-1,
    verbose=2
)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:   20.7s finished


In [39]:
print("CV AUCs:", log_cross_score)
print("Mean CV AUC:", log_cross_score.mean())

CV AUCs: [0.71478813 0.71317131 0.71444034]
Mean CV AUC: 0.7141332622386299


### ROC-AUC Interpretation

An AUC of approximately 0.71 indicates good ranking power for a credit risk model.

The model can meaningfully separate higher-risk borrowers from lower-risk borrowers, although default behavior remains influenced by multiple interacting factors.

In retail credit portfolios, AUC values between 0.65–0.75 are common for logistic scorecards.


In [40]:
log_pipe.fit(X_train,y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('numerical',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  Index(['loan_amnt', 'term', 'int_rate', 'installment', 'revol_util',
       'revol_bal', 'bc_util', 'all_util', 'total_rev_hi_lim',
       'bc_open_to_buy', 'max_bal_bc', 'total_bc_limit', 'total_bal_ex_mort'...
       'collections_12_mths_ex_med', 'tax_liens', 'annual_inc', 'emp_length',
       'credit_age'],
      dtype='object')),
                                                 ('categorical',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  Index(['purpose', 'verification_status', 'home_ownership', 'addr_state'], dtype='object'))])),
                ('model', LogisticRegression(max_iter=1000, random_state=42))])

In [41]:
test_prob = log_pipe.predict_proba(X_test)[:,1]

In [42]:
test_auc = roc_auc_score(y_test,test_prob)

In [43]:
print("Final Test AUC:", test_auc)

Final Test AUC: 0.7138217360305794


now our model gives probabilities for default or PD but it doesn't give us the financial decision so we define some features like:-
1. ead:- Exposure At Default(the amount exposed to the bank or lending company after default)
2. pd_score:- the probability score
3. actual default:- actual outcome
4. risk bucket:- segmenting pd_score in 5 sections to find the connection between pd_score and actual default
5. lgd :- loss given default
6. expected_loss:- the expected loss on an individual for the bank for lending money which is basically (ead * lgd * pd_score)

##  Probability Score Generation

Predicted probabilities from the Logistic Regression model are interpreted as the Probability of Default (PD).

A structured evaluation dataset is created containing:

- Predicted PD score
- Actual default outcome

This dataset enables risk segmentation, decile analysis, and Expected Loss simulation.


In [44]:
X_test_with_pd = X_test.copy()
X_test_with_pd['pd_score'] = test_prob
X_test_with_pd['actual_default'] = y_test.values

X_test_with_pd[['pd_score','actual_default']].head()

,pd_score,actual_default
1311262,0.130102,0
823606,0.118249,0
196043,0.151355,0
810959,0.141054,0
1157129,0.183177,0


##  Risk Segmentation

Borrowers are segmented into risk buckets based on predicted PD.

Risk buckets simulate how a financial institution might categorize customers into:

- Very Low Risk
- Low Risk
- Medium Risk
- High Risk
- Very High Risk

Segmentation enables differentiated credit policies across risk tiers.


In [45]:
X_test_with_pd['risk_bucket'] = pd.qcut(
    X_test_with_pd['pd_score'],
    q=5,
    labels=['Very Low','Low','Medium','High','Very High']
)

In [46]:
X_test_with_pd['risk_bucket'].value_counts()

,count
risk_bucket,
Very Low,53813
Very High,53813
Low,53812
Medium,53812
High,53812


### Bucket-Level Risk Analysis

For each risk bucket, the following metrics are evaluated:

- Average predicted PD across risk buckets

A well-performing model should demonstrate increasing observed default rates across higher risk buckets.


In [47]:
bucket_analysis = X_test_with_pd.groupby('risk_bucket')['actual_default'].mean()
bucket_analysis

/tmp/ipython-input-3765258614.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  bucket_analysis = X_test_with_pd.groupby('risk_bucket')['actual_default'].mean()


,actual_default
risk_bucket,
Very Low,0.059614
Low,0.116535
Medium,0.173567
High,0.249833
Very High,0.398584


##  Baseline Expected Loss Calculation

Expected Loss (EL) is calculated at the individual borrower level using:

\[
EL = PD * LGD * EAD
\]

Assumptions:

- LGD is assumed constant at 60%
- EAD is approximated using total revolving credit limit
- PD is derived from model predictions

The portfolio-level Expected Loss is obtained by aggregating borrower-level EL.


In [48]:
X_test_with_pd['ead'] = X_test_with_pd['total_rev_hi_lim']

In [49]:
lgd = 0.6  #assuming that loss given default is 60% as organization is able to recover 40% of the cost

In [50]:
X_test_with_pd['expected_loss'] = X_test_with_pd['pd_score']*X_test_with_pd['ead']*lgd

In [51]:
baseline_portfolio_el = X_test_with_pd['expected_loss'].sum()
baseline_portfolio_el

np.float64(884115475.5260327)

##  Credit Limit Adjustment Policy

A rule-based credit limit adjustment strategy is introduced.

Each risk bucket is assigned a credit limit multiplier:

- Lower-risk borrowers retain or receive higher limits
- Higher-risk borrowers receive reduced exposure

This simulates proactive risk management without fully rejecting customers.


In [52]:
def adjust_limit(bucket):
    if bucket == 'Very Low':
        return 1.10
    elif bucket == 'Low':
        return 1.05
    elif bucket == 'Medium':
        return 1.00
    elif bucket == 'High':
        return 0.80
    else:
        return 0.60

X_test_with_pd['limit_multiplier'] = (
    X_test_with_pd['risk_bucket'].apply(adjust_limit)
)


### Post-Adjustment Expected Loss

Using adjusted credit limits (new EAD), Expected Loss is recalculated:

\[
New EL = PD * LGD * ajusted EAD
\]

The portfolio-level impact is evaluated by comparing baseline EL with adjusted EL.


In [53]:
X_test_with_pd['new_ead'] = (
    X_test_with_pd['ead'] * X_test_with_pd['limit_multiplier'].astype(float)
)

X_test_with_pd['adjusted_EL'] = (
    X_test_with_pd['pd_score'] * lgd * X_test_with_pd['new_ead']
)

adjusted_portfolio_EL = X_test_with_pd['adjusted_EL'].sum()

print("Adjusted Portfolio EL:", adjusted_portfolio_EL)


Adjusted Portfolio EL: 723944600.9236706


##  Portfolio Impact Assessment

The comparison between baseline and adjusted Expected Loss shows the effectiveness of dynamic credit line adjustments.

A reduction in portfolio EL indicates improved capital efficiency and reduced credit risk exposure.

Percentage reduction quantifies the impact of the policy intervention.


In [54]:
reduction = baseline_portfolio_el - adjusted_portfolio_EL
reduction_per = (reduction/baseline_portfolio_el)*100

In [55]:
reduction_per

np.float64(18.1165106862385)

## Model Calibration & Policy Refinement
######(recommender:- chat gpt)

##  Model Evaluation

Beyond AUC, the model is evaluated using:

- Decile analysis
- Calibration comparison
- Confusion matrix
- Coefficient interpretation


In [56]:
X_test_with_pd['decile'] = pd.qcut(X_test_with_pd['pd_score'],q=10,labels=False,duplicates='drop')

In [57]:
calibration_table = (
    X_test_with_pd
    .groupby('decile')
    .agg(
        avg_predicted_pd=('pd_score', 'mean'),
        actual_default_rate=('actual_default', 'mean'),
        count=('actual_default', 'count')
    )
    .reset_index()
)

calibration_table


,decile,avg_predicted_pd,actual_default_rate,count
0,0,0.049590,0.042963,26907
1,1,0.082667,0.076266,26906
2,2,0.107842,0.104140,26906
3,3,0.131894,0.128930,26906
4,4,0.156941,0.163049,26906
5,5,0.184493,0.184085,26906
6,6,0.217274,0.226678,26906
7,7,0.260112,0.272987,26906
8,8,0.326131,0.336877,26906
9,9,0.477818,0.460289,26907


### Decile Analysis Interpretation

Default rates increase monotonically across risk deciles, confirming that the model effectively ranks borrowers by risk level.

Monotonicity is critical for credit policy design and risk segmentation.


##  Final Business Summary

- Model AUC: ~0.71  
- Monotonic decile risk separation achieved  
- Baseline Expected Loss calculated using PD × LGD × EAD  
- Credit limit adjustment policy implemented  
- Portfolio Expected Loss reduced by approximately 18%

This demonstrates how predictive modeling can directly inform credit strategy and improve portfolio-level risk management.


##  Deployment Simulation

In a production environment, the trained PD model would be:

- Saved as a serialized object (e.g., joblib/pickle)
- Integrated into a credit decision engine
- Used to generate PD scores in real time at loan origination

Credit limit adjustments would then be applied automatically based on predefined risk bucket policies.


In [58]:
import joblib
joblib.dump(log_pipe, "pd_model.pkl")

['pd_model.pkl']

In [66]:
loaded_model = joblib.load('pd_model.pkl')

# Test inference
sample_pd = loaded_model.predict_proba(X_test.iloc[:5])[:, 1]
print("Sample PD predictions:", sample_pd)
print('actual results:-\n',y_test[:5])

Sample PD predictions: [0.13010249 0.11824909 0.15135548 0.14105354 0.18317691]
actual results:-
 1311262    0
823606     0
196043     0
810959     0
1157129    0
Name: default, dtype: int32


In [67]:
def score_new_applicant(applicant_df, model):
    """
    Takes raw applicant dataframe (same structure as training data)
    Returns predicted PD
    """
    pd_score_new = model.predict_proba(applicant_df)[:, 1]
    return pd_score_new


## Model Monitoring Considerations

In a real-world credit risk environment, ongoing monitoring is critical.

Key monitoring metrics would include:

- Population Stability Index (PSI)
- AUC drift over time
- Calibration drift
- Portfolio default rate shifts

Periodic retraining would be required if significant distribution shift is observed.


In [68]:
# Simulating new incoming batch (using test as proxy)
y_new_pred = loaded_model.predict_proba(X_test)[:, 1]

auc_new = roc_auc_score(y_test, y_new_pred)

print("Current Production AUC:", auc_new)


Current Production AUC: 0.7138217360305794


In [71]:
def calculate_psi(expected, actual, buckets=10):
    """
    Calculate Population Stability Index (PSI)
    expected: training PD array
    actual: new PD array
    """

    # Convert to pandas Series (important)
    expected = pd.Series(expected)
    actual = pd.Series(actual)

    # Create bins using expected distribution
    quantiles = np.linspace(0, 1, buckets + 1)
    breakpoints = expected.quantile(quantiles).values
    breakpoints[0] = -np.inf
    breakpoints[-1] = np.inf

    # Bin both distributions
    expected_bins = pd.cut(expected, bins=breakpoints)
    actual_bins = pd.cut(actual, bins=breakpoints)

    # Calculate distributions
    expected_dist = expected_bins.value_counts().sort_index() / len(expected)
    actual_dist = actual_bins.value_counts().sort_index() / len(actual)

    # Replace zeros to avoid log(0)
    expected_dist = expected_dist.replace(0, 1e-6)
    actual_dist = actual_dist.replace(0, 1e-6)

    psi = np.sum((actual_dist - expected_dist) *
                 np.log(actual_dist / expected_dist))

    return psi


In [72]:
train_pd = loaded_model.predict_proba(X_train)[:, 1]
test_pd = loaded_model.predict_proba(X_test)[:, 1]

psi_value = calculate_psi(train_pd, test_pd)

print("PSI between train and test:", psi_value)

PSI between train and test: 3.324077506108574e-05


In [80]:
def credit_decision_workflow(applicant_df, model, lgd=0.6):

    pd_score = float(model.predict_proba(applicant_df)[0, 1])

    if pd_score < 0.05:
        multiplier = 1.1
        bucket = "Very Low"
    elif pd_score < 0.10:
        multiplier = 1.0
        bucket = "Low"
    elif pd_score < 0.20:
        multiplier = 0.8
        bucket = "Medium"
    else:
        multiplier = 0.6
        bucket = "High"

    original_limit = float(applicant_df["revol_bal"].iloc[0])
    adjusted_limit = float(original_limit * multiplier)
    expected_loss = float(pd_score * lgd * adjusted_limit)

    return {
        "Probability of Default": round(pd_score, 4),
        "Risk Bucket": bucket,
        "Original Credit Limit": round(original_limit, 2),
        "Adjusted Credit Limit": round(adjusted_limit, 2),
        "Expected Loss ($)": round(expected_loss, 2)
    }


In [82]:
result = credit_decision_workflow(X_test.iloc[[0]], loaded_model)

pd.DataFrame(result.items(), columns=["Metric", "Value"])

,Metric,Value
0,Probability of Default,0.1301
1,Risk Bucket,Medium
2,Original Credit Limit,20816.0
3,Adjusted Credit Limit,16652.8
4,Expected Loss ($),1299.94


## Operational Workflow Summary

End-to-end system workflow:

1. Borrower applies for credit.
2. Model generates PD score.
3. Borrower assigned to risk bucket.
4. Credit limit multiplier applied.
5. Expected Loss computed at portfolio level.
6. Risk strategy monitored and updated periodically.

This completes the end-to-end credit risk management cycle.


## Project Conclusion

This project demonstrates a full credit risk lifecycle:

- Data preparation
- Risk modeling
- Risk segmentation
- Credit policy simulation
- Portfolio risk reduction
- Deployment and monitoring considerations

The approach illustrates how predictive analytics can directly support strategic credit decision-making.
